In [1]:
import sys
# Assuming we are in the notebook directory add this so that we can import the library
sys.path.append('..')

import numpy as np
import dask
from dask.dot import dot_graph
from elfi import *
from elfi.examples.ma2 import MA2, autocov, distance
from distributed import Client
from functools import partial

import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
# Setup distributed
# client = Client()
# dask.set_options(get=client.get)

In [3]:
n = 100
t1_0 = 0.6
t2_0 = 0.2

# Set up observed data y
latents = np.random.randn(n+2)
y = MA2(n, t1_0, t2_0, n_sim=1, latents=latents)

# Plot
# plt.figure(figsize=(11, 6));
# plt.plot(np.arange(0,n),y[0,:]);
# plt.scatter(np.arange(-2,n), latents);

In [4]:
# Set up the simulator
simulator = partial(MA2, n)

# Set up autocovariance summaries
ac1 = partial(autocov, 1)
ac2 = partial(autocov, 2)

# Specify the graphical model
t1 = Prior('t1', 'uniform', 0, 1)
t2 = Prior('t2', 'uniform', 0, 1)
Y = Simulator('MA2', simulator, t1, t2, observed=y)
S1 = Summary('S1', ac1, Y)
S2 = Summary('S2', ac2, Y)
d = Discrepancy('d', distance, S1, S2, store=MemoryStore())

In [5]:
# Specify the number of simulations and set up rejection sampling
N = 10000
rej = Rejection(N, d, [t1, t2], 10000)

In [6]:
# Time and run the simulator in parallel
%time result = rej.infer(quantile=0.1)

CPU times: user 636 ms, sys: 81.8 ms, total: 718 ms
Wall time: 949 ms


 /home/lintusj1/.conda/envs/abc4py/lib/python3.5/site-packages/distributed/worker.py:802: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future
 /home/lintusj1/.conda/envs/abc4py/lib/python3.5/site-packages/distributed/worker.py:802: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future


In [7]:
%time result = rej.infer(quantile=0.1)

distributed.utils - ERROR - only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Traceback (most recent call last):
  File "/home/lintusj1/.conda/envs/abc4py/lib/python3.5/site-packages/distributed/utils.py", line 120, in f
    result[0] = yield gen.maybe_future(func(*args, **kwargs))
  File "/home/lintusj1/.conda/envs/abc4py/lib/python3.5/site-packages/tornado/gen.py", line 1015, in run
    value = future.result()
  File "/home/lintusj1/.conda/envs/abc4py/lib/python3.5/site-packages/tornado/concurrent.py", line 237, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 3, in raise_exc_info
  File "/home/lintusj1/.conda/envs/abc4py/lib/python3.5/site-packages/tornado/gen.py", line 1021, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/lintusj1/.conda/envs/abc4py/lib/python3.5/site-packages/distributed/client.py", line 794, in _gather
    d['traceback'])
  File "/home/lintusj1/.conda/envs/abc4py/

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
result

In [ ]:
d._store._delayed_outputs[0]

In [ ]:
d[0:10].compute()